In [1]:
import os
import pandas as pd
import numpy as np
import pathlib
import pickle
import copy
import matplotlib.pyplot as plt
project_path ='D:/IJPR/rand/c_impact_2/'

# effect of num of jobs

In [ ]:
n = 10 # num of jobs
delta_mu = 4 # control lb of mean processing time
delta_r = 0.05 # control ub of the release time
delta_ep = 1.5 # control the upper bound of the mad
S_train = 30
iterations = 10
delta_mad_r = 0.3

In [ ]:
it = 0
ins = 0
Seed = 0
full_path = project_path + '/data/seed='+str(Seed)+'/ins='+str(ins) + '/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r='+str(delta_mad_r)
tft_df = pd.read_csv(full_path+'tft.csv')

In [ ]:
c_set = np.arange(0,1,0.05)

In [ ]:
[S,N_c] = tft_df.shape

In [ ]:
mean0 = (tft_df.mean(axis = 0).values)[1:N_c]

In [ ]:
tft_avg_all = pd.DataFrame()
tft_quan75_all = pd.DataFrame()
tft_quan95_all = pd.DataFrame()
for ins in range(10):
    Seed = ins
    tft_avg = np.ones((iterations,len(mean0)))
    tft_quan75 = np.ones((iterations,len(mean0)))
    tft_quan95 = np.ones((iterations,len(mean0)))
    for it in range(iterations):
        full_path = project_path + '/data/seed='+str(Seed)+'/ins='+str(ins) +'/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r='+str(delta_mad_r)
        # print(full_path)
        tft_df = pd.read_csv(full_path+'tft.csv')
        tft_avg[it,:] = (tft_df.mean(axis = 0).values)[1:N_c]
        tft_quan75[it,:] = (tft_df.quantile(axis = 0,q=0.75).values)[1:N_c]
        tft_quan95[it,:] = (tft_df.quantile(axis = 0,q=0.95).values)[1:N_c]
    tft_avg_all['ins='+str(ins)] = np.mean(tft_avg,axis = 0)
    tft_quan75_all['ins='+str(ins)] = np.mean(tft_quan75,axis = 0)
    tft_quan95_all['ins='+str(ins)] = np.mean(tft_quan95,axis = 0)

In [ ]:
tft_avg_all.to_csv(project_path + '/data/tft_avg_all.csv')
tft_quan75_all.to_csv(project_path + '/data/tft_quan75_all.csv')
tft_quan95_all.to_csv(project_path + '/data/tft_quan95_all.csv')

In [ ]:
tft_avg_all

In [ ]:
tft_avg_ratio_all = copy.deepcopy(tft_avg_all)
tft_quan75_ratio_all = copy.deepcopy(tft_quan75_all)
tft_quan95_ratio_all = copy.deepcopy(tft_quan95_all)
for i in range(22):
    tft_avg_ratio_all.iloc[i] = tft_avg_all.iloc[i]/tft_avg_all.iloc[0]
    tft_quan75_ratio_all.iloc[i] = tft_quan75_all.iloc[i]/tft_quan75_all.iloc[0]
    tft_quan95_ratio_all.iloc[i] = tft_quan95_all.iloc[i]/tft_quan95_all.iloc[0]

In [ ]:
tft_avg_ratio_all.to_csv(project_path + '/data/tft_avg_ratio_all.csv')
tft_quan75_ratio_all.to_csv(project_path + '/data/tft_quan75_ratio_all.csv')
tft_quan95_ratio_all.to_csv(project_path + '/data/tft_quan95_ratio_all.csv')

# impact of $\delta_{\epsilon}$

In [ ]:
tft_avg_all = pd.DataFrame()
tft_quan75_all = pd.DataFrame()
tft_quan95_all = pd.DataFrame()
delta_ep_all = [1.5,2,2.5]
ins = 0
for delta_ep in delta_ep_all:
    Seed = ins
    tft_avg = np.ones((iterations,len(mean0)))
    tft_quan75 = np.ones((iterations,len(mean0)))
    tft_quan95 = np.ones((iterations,len(mean0)))
    for it in range(iterations):
        full_path = project_path + '/delta_ep/data/seed='+str(Seed)+'/ins='+str(ins) +'/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r='+str(delta_mad_r)
        tft_df = pd.read_csv(full_path+'tft.csv')
        tft_avg[it,:] = (tft_df.mean(axis = 0).values)[1:N_c]
        tft_quan75[it,:] = (tft_df.quantile(axis = 0,q=0.75).values)[1:N_c]
        tft_quan95[it,:] = (tft_df.quantile(axis = 0,q=0.95).values)[1:N_c]
    tft_avg_all['delta_ep='+str(delta_ep)] = np.mean(tft_avg,axis = 0)
    tft_quan75_all['delta_ep='+str(delta_ep)] = np.mean(tft_quan75,axis = 0)
    tft_quan95_all['delta_ep='+str(delta_ep)] = np.mean(tft_quan95,axis = 0)

In [ ]:
tft_avg_all.to_csv(project_path + '/delta_ep/data/tft_avg_all.csv')
tft_quan75_all.to_csv(project_path + '/delta_ep/data/tft_quan75_all.csv')
tft_quan95_all.to_csv(project_path + '/delta_ep/data/tft_quan95_all.csv')

In [ ]:
tft_avg_ratio_all = copy.deepcopy(tft_avg_all)
tft_quan75_ratio_all = copy.deepcopy(tft_quan75_all)
tft_quan95_ratio_all = copy.deepcopy(tft_quan95_all)
for i in range(27):
    tft_avg_ratio_all.iloc[i] = tft_avg_all.iloc[i]/tft_avg_all.iloc[0]
    tft_quan75_ratio_all.iloc[i] = tft_quan75_all.iloc[i]/tft_quan75_all.iloc[0]
    tft_quan95_ratio_all.iloc[i] = tft_quan95_all.iloc[i]/tft_quan95_all.iloc[0]

In [ ]:
tft_avg_ratio_all.to_csv(project_path + '/delta_ep/data/tft_avg_ratio_all.csv')
tft_quan75_ratio_all.to_csv(project_path + '/delta_ep/data/tft_quan75_ratio_all.csv')
tft_quan95_ratio_all.to_csv(project_path + '/delta_ep/data/tft_quan95_ratio_all.csv')

# Effect of r mad

In [ ]:
project_path = 'D:/IJPR/rand/mad_r_2/'
Seed = 0
it = 0
ins = 0
full_path = project_path + '/data/seed='+str(Seed)+'/ins='+str(ins) + '/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r=0.3'
tft_df = pd.read_csv(full_path+'tft.csv')


In [ ]:
tft_df

In [ ]:
[N_s,N_c] = np.shape(tft_df.values)
tft_df.columns
col_names = ['det', 'saa', 'wass_488.14498736448166']

In [ ]:
n_all = [10,15,20]
df_final = {}
for n in n_all:
    ins = 0
    Seed = 0
    df_set = {}
    delta_all = [1.5,2,2.5]
    for delta in delta_all:
        it = 0
        full_path = project_path + '/data/seed='+str(Seed)+'/ins='+str(ins) + '/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r=0.3'
        tft_df = pd.read_csv(full_path+'tft.csv')
        df = pd.DataFrame(np.zeros((N_s * 3,3)))
        col_vector = tft_df.columns.values
        col_names = ['DET', 'SAA','W=0.5']
        col_vector = ['det', 'saa', col_vector[3]]
        df.columns = ['tft','appro','delta']
        for i in range(3):
            df['tft'][i*N_s:(i+1)*N_s] = tft_df[col_vector[i]]
            df['appro'][i*N_s:(i+1)*N_s] = np.asarray([col_names[i] for ind in range(N_s)])
            df['delta'][i*N_s:(i+1)*N_s] = np.ones(N_s)*delta
        df_set[delta] = copy.deepcopy(df)
    df_final[n] = pd.concat([df_set[delta] for delta in delta_all])


In [ ]:
import seaborn as sns
def plot_group_box(df_final,n_all,fixed_name):
    
    # Plot grouped box plot
    font_size = 16
    tick_size = 12
    line_width = 1
    my_pal = { "DET":"m","SAA": "b", "W=0.5": "r"}
    # my_pal = { "Forest":"m","RobustTreeAffine": "r", "RobustStatic": "b"}
    
    num_n = len(n_all)
    fig,ax = plt.subplots(1,num_n,figsize=(12, 8))
    for i in range(num_n):
        graphic = sns.boxplot(x=fixed_name, y='tft',hue = 'appro', data=df_final[n_all[i]],palette=my_pal,\
                                width = 0.75,showmeans = True,meanline = True, \
                                meanprops = {'linestyle':'--','color':'gold','linewidth':line_width},\
                                showfliers = False,\
                                medianprops = {'linestyle':'-','color':'k','linewidth':line_width},
                                linewidth = line_width,ax=ax[i])
        graphic.set_xlabel(r'$\delta_{\sigma}^{r}$',fontsize = font_size)
        graphic.xaxis.set_tick_params(labelsize=tick_size)
        if i == 0:
            graphic.set_ylabel('Total completion time',fontsize = font_size+2)
        else:
            graphic.set_ylabel('')
        graphic.yaxis.set_tick_params(labelsize=tick_size)
        graphic.set_title('n='+str(n_all[i]))
        # graphic.set_yscale('log')
        sns.color_palette('pastel')
        if i == 0:
            graphic.legend([])
        else:
            # graphic.legend(n_cols = 2)
            graphic.legend(loc=7,fontsize = font_size, bbox_to_anchor=(1.4,0.5),borderaxespad = 0.)
    plt.subplots_adjust(hspace=0.2,wspace=0.3)
    plt.savefig('impact_mad_r.pdf')
    plt.show()


In [ ]:
n_all = [10,20]
plot_group_box(df_final,n_all,'delta')

# effect of mad p

In [3]:
n = 10 # num of jobs
delta_mu = 4 # control lb of mean processing time
delta_r = 0.05 # control ub of the release time
delta_ep = 1.5 # control the upper bound of the mad
S_train = 30
S_test = 10000
quan_low = 0.1
quan_bar = 0.9         

delta_mad_r = 0.3

In [4]:
project_path = 'D:/IJPR/rand/mad_p_2/'
Seed = 0
it = 0
ins = 0
full_path = project_path + '/data/seed='+str(Seed)+'/ins='+str(ins) + '/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r=0.3'
tft_df = pd.read_csv(full_path+'tft.csv')


In [6]:
[N_s,N_c] = np.shape(tft_df.values)
tft_df.columns

Index(['Unnamed: 0', 'det', 'saa', 'wass_811.0590717990444'], dtype='object')

In [7]:
n_all = [10,15,20]
df_final = {}
for n in n_all:
    ins = 0
    Seed = 0
    df_set = {}
    delta_all = [1.5,2,2.5]
    for delta in delta_all:
        it = 0
        full_path = project_path + '/data/seed='+str(Seed)+'/ins='+str(ins) + '/it='+str(it)+'/S_train='+str(S_train)+'/n='+str(n)+'delta_mu='+str(delta_mu)+'delta_r='+str(delta_r)+'delta_ep='+str(delta_ep)+'delta_mad_r=0.3'
        tft_df = pd.read_csv(full_path+'tft.csv')
        df = pd.DataFrame(np.zeros((N_s * 3,3)))
        col_vector = tft_df.columns.values
        col_names = ['DET', 'SAA','W=0.5']
        col_vector = ['det', 'saa', col_vector[3]]
        df.columns = ['tft','appro','delta']
        for i in range(3):
            df['tft'][i*N_s:(i+1)*N_s] = tft_df[col_vector[i]]
            df['appro'][i*N_s:(i+1)*N_s] = np.asarray([col_names[i] for ind in range(N_s)])
            df['delta'][i*N_s:(i+1)*N_s] = np.ones(N_s)*delta
        df_set[delta] = copy.deepcopy(df)
    df_final[n] = pd.concat([df_set[delta] for delta in delta_all])


C:\Users\xunzhang\AppData\Local\Temp\ipykernel_4120\2073828279.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta'][i*N_s:(i+1)*N_s] = np.ones(N_s)*delta
C:\Users\xunzhang\AppData\Local\Temp\ipykernel_4120\2073828279.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tft'][i*N_s:(i+1)*N_s] = tft_df[col_vector[i]]
C:\Users\xunzhang\AppData\Local\Temp\ipykernel_4120\2073828279.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [ ]:
n_all = [10,20]
plot_group_box(df_final,n_all,'delta')